# Zip folder unpacking script
This script will do the following operations in order:
Scan a folder
create new calculation folder
Loop the following actions for each homer script


    Unseal homer template file (.homer -> .zip)
    
    
    Unpack zip file (.zip -> normal directory)
    
    
    Open 'problem' file
    
    
    ####### NOT FOR THIS SCRIPT modify 'problem' file
    
    
    Repack zip file (normal directory -> .zip)
    
    
    Reseal homer file (.zip -> .homer) 
        ->[FOR THIS STEP TO FUNCTION PROPERLY, THE JUPYTER NOTEBOOK MUST BE EXECUTED ONE DIRECTORY HIGHER]
    
    
    store updated input file in calculation folder
    
    
    end 

In [2]:
# Script initialization, make sure to set working directory
import os
import numpy as np 
import zipfile
import shutil
import zlib
import pathlib 
import subprocess
import pandas as pd
from lxml import etree, objectify
folder = 'd:/repo/homer/ClimateStudies/ProductionCases/80000'

In [3]:
# Unsealing HOMER template files (.homer -> .zip)
files = [f for f in os.listdir(folder) if f.endswith('.homer')]
n_files = np.size(files)
display(files)
for i in range(len(files)):
    file = files[i]
    base = os.path.splitext(file)[0]
    os.rename(file, base + '.zip')

['Case14AnchorageDfc.homer',
 'Case14EielsonAFBDfC.homer',
 'Case14MedfordORCsa.homer']

In [4]:
# Unpack zip file (.zip -> normal directory)
files = [f for f in os.listdir(folder) if f.endswith('.zip')]
n_files = np.size(files)
for i in range(len(files)):
    file = files[i]
    base = os.path.splitext(file)[0]
    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall(base + 'TEMP')
    os.remove(file)
# Opening each individiual 'problem' file after extraction


In [5]:
# Pulling Case 1, Case 5 Data for Each File
files = os.listdir(folder)
output_df = {} #creates empty dictionary
names = [] #creates empty list for future name storage
for file in files:
    if file.endswith('.xlsx'):
        xls = pd.ExcelFile(folder + '/' + file) #reads file
        names = names + xls.sheet_names
        for sheet_name in xls.sheet_names: #pulls individual sheets
            output_df[sheet_name] = xls.parse(sheet_name) #puts each sheet into a dictionary w/key "sheet_name"
#clean_data = pd.DataFrame(columns=['CaseNum','Cost/COE\n($)','CaseName'])
#df = output_df[sheet_name]
#clean_data['CaseNum'] = df.loc[:,'CaseNum']
#clean_data['Cost/COE\n($)'] = df.loc[:,'Cost/COE\n($)']
#location_names = [s.replace(df.loc[:,'Unnamed: 0']
#display(type(df.loc[:,'Unnamed: 0']))
##clean_data['CaseName'] = location_names[5:]
##clean_data.set_index(['CaseName'],drop=True,inplace=True)
##clean_data = clean_data.sort_values(by=['CaseName'])
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    display(clean_data)

In [6]:
# (Within normal directory) Edit the .xml files for each HOMER problem.
files = [f for f in os.listdir(folder) if f.endswith('TEMP')]
display(files)
# paths to change as key, new value as value
new_data = {'/HomerProblem/FuelResources/Fuel/Name': 'CCM FUEL',
            '/HomerProblem/FuelResources/Fuel/FuelCostSensitivity/decimal': '0.00',
            '/HomerProblem/Generators/Generator/CostMatrix/Cost/Initial': '85000000',
            '/HomerProblem/Generators/Generator/CostMatrix/Cost/Quantity': '1000' ,
            '/HomerProblem/Generators/Generator/CostMatrix/Cost/CapitalCostDetail/CostCalculatorItem/UnitCost': '85000' ,
            '/HomerProblem/Generators/Generator/CostMatrix/Cost/Replacement': '3550000',
            '/HomerProblem/Generators/Generator/CostMatrix/Cost/ReplacementCostDetail/CostCalculatorItem/Quantity': '1000' ,
            '/HomerProblem/Generators/Generator/CostMatrix/Cost/ReplacementCostDetail/CostCalculatorItem/UnitCost': '3550000' ,
            '/HomerProblem/Generators/Generator/LifetimeSensitivity/decimal': '43830' ,
            '/HomerProblem/Generators/Generator/CostMatrix/Cost/Operating': '58.33',
            '/HomerProblem/SimulationParameters/Economics/ProjectLifetimeSensitivity/decimal': '60',
            '/HomerProblem/SimulationParameters/SystemControl/Optimizer/FocusFactor': '2'}
   
for i in range(len(files)):
    file = files[i]
    problems = etree.parse(file + '/problem')
    #os.remove(file + '/calculated-problem') #deletes old homer savestate
    for path, value in new_data.items():
       # if 'CapacityOptimization' in path:#not necessary currently, but can be used to differentiate values later
       #     problems.xpath(path)[0].text = value
       #     newfile = etree.tostring(problems,encoding='utf-8',method='xml', xml_declaration='<?xml version="1.0" encoding="utf-8"?>')
       #     newfile = str(newfile,'utf-8')#otherwise use utf-8
       # else: 
            #print(path, ' => ', problems.xpath(path)[0].text)
        problems.xpath(path)[0].text = value
        newfile = etree.tostring(problems,encoding='US-ASCII',method='xml', xml_declaration='<?xml version="1.0" encoding="utf-8"?>')
        newfile = str(newfile,'US-ASCII')#otherwise use utf-8
    with open(file + '/problem','w') as f:
    #   # newfile = etree.tostring(new)
         f.write(newfile)

['Case14AnchorageDfcTEMP', 'Case14EielsonAFBDfCTEMP', 'Case14MedfordORCsaTEMP']

In [7]:
#Repack zip file (normal directory -> .zip)
## THIS STEP CURRENTLY BREAKS THE HOMER FILE
## Try using zipcode compression 
files = [f for f in os.listdir(folder) if f.endswith('TEMP')]
display(files)
for i in range(len(files)):
    file = files[i]
    directory = pathlib.Path(file)
    basefile = file.replace('TEMP','')
    basefile = basefile.replace('Case14','Case15')
    subprocess.run(["powershell.exe", "Compress-Archive -Path " + file + "\* -DestinationPath " + basefile + " -Force"])
    shutil.rmtree(file) #deletes old temp folders
    

['Case14AnchorageDfcTEMP', 'Case14EielsonAFBDfCTEMP', 'Case14MedfordORCsaTEMP']

In [8]:
# Reseal homer file (.zip -> .homer)
files = [f for f in os.listdir(folder) if f.endswith('.zip')]
n_files = np.size(files)
type(files)
for i in range(len(files)):
    file = files[i]
    base = os.path.splitext(file)[0]
    os.rename(file, base + '.homer')